In [ ]:
import numpy as np
import pandas as pd
import json
import os
from pathlib import Path
import glob
from util import pil_imread
from tqdm import tqdm

import tifffile as tf
from skimage.transform import rescale
import skimage.util as skiu

import cv2

In [ ]:
def stiched_image(img_path, output_file_name=None, imgchn=0, px_size=0.11):
    """
    author: Lincoln Ombelets (modified by Katsuya Colon)
    
    getStagePositions:
    Parse a MicroManager position list to return the X,Y,Z
    positions and names of each position.
    """

    '''
    Stage position list

    The stage position list gives the bottom left corner coordinate of each position
    The X coordinate is horizontal position (+ is right), the Y coordinate is vertical position (+ is up)
    The (0, 0) of a Numpy array is the top left corner when plotted as an image.
    The first coordinate is row i.e. vertical position - so Y - where + is down
    The second coordinate is column i.e. horizontal position - so X - where + is right
    Thus overall, to transform between stage coordinates and Numpy array indices, swap X and Y then flip Y
    
    Parameters
    -----------
    img_path = directory to aligned segmentation image
    imgchn = which channel to stitch
    output_file = prefix of output file name
    px_size = interpixel distance on camera
    '''
    
    #ask for max project or not
    max_proj = input("Do you want to max project image (y/n): ")
    if max_proj == "n":
        z = input("Which z layer do you want (number): ")
    
    #write output directory
    parent = Path(img_path)
    while "pyfish_tools" not in os.listdir(parent):
         parent = parent.parent
    output_file = parent /"pyfish_tools"/"output"/ "stiched_img" / (output_file_name + "_stitched.tif")
    output_file.parent.mkdir(parents=True, exist_ok=True)
    output_file = str(output_file)
    large_file = parent / "pyfish_tools"/"output"/ "stiched_img" / (output_file_name + "_large_stitched.tif")
    large_file.parent.mkdir(parents=True,exist_ok=True)
    large_file = str(large_file)

    #collect pos file
    filename = str(list(parent.glob("*.pos"))[0])
    
    #collect metadata
    with open(filename) as f:
        content = json.load(f)

    positions = []

    for pos in content['POSITIONS']:

        z_name = pos['DEFAULT_Z_STAGE']
        xy_name = pos['DEFAULT_XY_STAGE']
        
        gridcol = pos.get('GRID_COL', -1)
        gridrow = pos.get('GRID_ROW', -1)

        posinfo = {
            'label': pos['LABEL'],
            'gridrow': gridrow,
            'gridcol': gridcol
        }
        

        for dev in pos['DEVICES']:

            if dev['DEVICE'] == z_name:
                posinfo['z'] = dev['X']

            if dev['DEVICE'] == xy_name:
                posinfo['x'] = dev['X']
                posinfo['xpx'] = round(dev['X']/px_size)
                posinfo['y'] = dev['Y']
                posinfo['ypx'] = round(dev['Y']/px_size)

        positions.append(posinfo)

    pos_df = pd.DataFrame(positions)

    '''(0, 0) in the stage position list is bottom left, and X and Y correspond to horizontal and vertical directions respectively.
    +X is to the right, +Y is up.
    So the Y axis is flipped
    And stage X and Y correspond to axes 1 and 0 of a 2D image in Numpy row-major order
    So axis 0 needs to flip
    '''

    botleft = (pos_df['xpx'].min(), pos_df['ypx'].min())
    topright = (pos_df['xpx'].max(), pos_df['ypx'].max())

    pos_df['xind'] = pos_df['xpx'] - botleft[0]
    pos_df['yind'] = pos_df['ypx'] - botleft[1]
    pos_df['gridrow'] = pos_df['yind'] // 2048
    pos_df['gridcol'] = pos_df['xind'] // 2048

    pos_df.sort_values(by=['gridrow', 'gridcol'])

    pos_df['position'] = pos_df.index 


    topright_index = (topright[0]-botleft[0]+2049, topright[1]-botleft[1]+2049)
    topright_index[0] * topright_index[1] * 2 / 1e6

    scale_factor = 1
    scaled_size = (int(np.ceil(topright_index[1]/scale_factor)), int(np.ceil(topright_index[0]/scale_factor)))
    full_image = np.zeros(scaled_size, dtype=np.uint16)


    pos0_im = []

    for i, p in tqdm(enumerate(pos_df.to_dict(orient='records'))):
        posim = img_path + '/MMStack_Pos' + str(p['position']) + '.ome.tif'
    
        imarr = pil_imread(posim, swapaxes=True)
        if len(imarr.shape) == 3:
            imarr = imarr.reshape(1, imarr.shape[0],imarr.shape[1],imarr.shape[2])
        if max_proj == "y":
            imarr = np.max(imarr,axis=0)
            imarr = imarr[imgchn]
        else:
            imarr = imarr[int(z)][imgchn]

        # The indices, from top left of the big image, of the bottom right corner of this tile
        im_botleft = ((scaled_size[0] - p['yind']) // scale_factor, 
                      (p['xind']) // scale_factor)
        #print(im_botleft, im_botleft[0]-imarr.shape[0])
    
        # We must index from the top left, so subtract the height of the image from
        # the bottom left coordinate.
        full_image[im_botleft[0]-imarr.shape[0]:im_botleft[0], 
                im_botleft[1]:im_botleft[1]+imarr.shape[1]] = skiu.img_as_uint(imarr)
    
    print("writing image...")
    #write large file
    tf.imwrite(large_file, full_image, compression='DEFLATE')
    
    #write small file
    scale_percent = 50 # percent of original size
    width = int(full_image.shape[1] * scale_percent / 100)
    height = int(full_image.shape[0] * scale_percent / 100)
    dim = (width, height)
    
    resized = cv2.resize(full_image, dim, interpolation = cv2.INTER_NEAREST)
    tf.imwrite(output_file, resized, compression='DEFLATE')
    print("Done...")

In [ ]:
#src: directory of your images
src = ""
stiched_image(src, imgchn=0, output_file_name="grft", px_size=0.11)